In [37]:
import sys
sys.path.append('..')

import numpy as np
from src.helpers import load_csv_data, standardize, remove_incomplete_columns, predict_labels, create_csv_submission
from src.logistic.implementations import logistic_regression
from src.logistic.loss import compute_loss
from src.logistic.not_req_impl import reg_logistic_regression, gradient_descent_step
from src.logistic.gradient import compute_gradient
from src.split import split_data

from src.logistic.sigmoid import sigmoid

%reload_ext autoreload
%autoreload 2

In [12]:
# Import data
y, x_raw, ids = load_csv_data('../data/train.csv')

In [14]:
# Clean the data
x, kept_columns = remove_incomplete_columns(x_raw)

In [15]:
kept_columns

array([ True,  True,  True,  True, False, False, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False,  True])

In [16]:
# Transform y for logistic regression
y[np.where(y == -1)] = 0

In [17]:
# Standardize x and add the 1s column
x, mean_x, std_x = standardize(x)
tx = np.c_[np.ones((y.shape[0], 1)), x]

In [18]:
# Split the data
tx_train, y_train, tx_test, y_test = split_data(tx, y, 0.8)

In [ ]:
initial_w = np.zeros((tx.shape[1], 1))

w, tr_loss = reg_logistic_regression(y, tx, initial_w, 0, 10000, 0.01, method='sgd')

Current iteration=0, loss=[173286.79513999]
||d|| = 0.9127409134069878
Current iteration=100, loss=[154958.09630973]
||d|| = 0.8142576999435608
Current iteration=200, loss=[155036.53692035]
||d|| = 0.9543905620823595
Current iteration=300, loss=[151956.9698921]
||d|| = 0.7003121916500041
Current iteration=400, loss=[152171.16631661]
||d|| = 0.7138316120731767
Current iteration=500, loss=[148891.22256581]
||d|| = 1.9909493067513162
Current iteration=600, loss=[148423.79372522]
||d|| = 0.6561252935664129


In [44]:
x_sgd = np.array([1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1]).reshape((1, -1))
y_sgd = np.array([-1])
w_sgd = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

%timeit compute_gradient(y_sgd, x_sgd, w_sgd)

10.4 µs ± 67.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [47]:
%timeit gradient_descent_step(y_sgd, x_sgd, w_sgd, 0.01)

41.5 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
te_loss = compute_loss(y_test, tx_test, w)
print(te_loss)

0.00560667986024832


In [10]:
y_sub, x_sub_raw, ids_sub = load_csv_data('../data/test.csv')

In [11]:
x_sub = x_sub_raw[:, kept_columns]
tx_sub = np.c_[np.ones((y_sub.shape[0], 1)), x_sub]
y_sub = predict_labels(w, tx_sub)

In [12]:
create_csv_submission(ids_sub, y_sub, '../submissions/10-20.16-01.csv')